In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "sd-legacy/stable-bdiffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to("cpu")

prompt = "a cat"
image = pipe(prompt).images[0]  
    
image.save("cat.png")


### Dont run this model on a raspberry pi (you can still try). 
Either, something is not done properly, or it does not support ``torch.float32``, which is needed for CPU-execution.

In [ ]:
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
from diffusers.utils import load_image
import PIL

import torch

pipe_prior = KandinskyV22PriorPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float32
)
pipe_prior.to("cpu")

img1 = load_image(
    "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main" "/kandinsky/cat.png"
)

img2 = load_image(
    "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main" "/kandinsky/starry_night.jpeg"
)

# add all the conditions we want to interpolate, can be either text or image
images_texts = ["a cat", img1, img2]

# specify the weights for each condition in images_texts
weights = [0.3, 0.3, 0.4]

# We can leave the prompt empty
prompt = ""
prior_out = pipe_prior.interpolate(images_texts, weights)

pipe = KandinskyV22Pipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float32)
pipe.to("cpu")

image = pipe(**prior_out, height=768, width=768).images[0]

image.save("starry_cat.png")
